In [ ]:
import numpy as np

In [ ]:
np.random.seed(1234)

In [ ]:
NB_BOIDS = 50
MIN_DIST = 0.01
Xmin = 0
Xmax = 1
Ymin = 0
Ymax = 1
Vxmax = 0.01
Vymax = 0.01

In [ ]:
# for boids with limited FoV, set to a small value (like 0.03)
# else set it to a big value (like 10)
PERCEPTION_RADIUS = 0.03 

In [ ]:
X = np.random.rand(1, NB_BOIDS)
Y = np.random.rand(1, NB_BOIDS) 
Vx = np.random.rand(1, NB_BOIDS) * Vxmax * 2 - Vxmax
Vy = np.random.rand(1, NB_BOIDS)* Vymax * 2 - Vymax

In [ ]:
def update():
    # based on https://vergenet.net/~conrad/boids/pseudocode.html
    # with addition of limited perception radius by me
    global X, Y, Vx, Vy
    
    XX = np.transpose(X) - X
    YY = np.transpose(Y) - Y
    SELF = np.identity(NB_BOIDS) == 1

    DIST = np.sqrt(XX**2 + YY**2)

    NEIGHBOURS = (DIST < PERCEPTION_RADIUS) & (~SELF)
    NB_NEIGHBOURS = np.sum(NEIGHBOURS, axis=1)
    NB_NEIGHBOURS = np.maximum(NB_NEIGHBOURS, 1) # avoid division by zeros when no neighbours (if FoV)

    # rule 1
    CENTER_OF_MASS_X = np.sum(X * NEIGHBOURS, 1) / NB_NEIGHBOURS
    CENTER_OF_MASS_Y = np.sum(Y * NEIGHBOURS, 1) / NB_NEIGHBOURS
    CENTER_OF_MASS_X = (CENTER_OF_MASS_X - X) / 100
    CENTER_OF_MASS_Y = (CENTER_OF_MASS_Y - Y) / 100

    # rule 2
    CLOSE_NEIGHBOURS = (DIST < MIN_DIST) & (~SELF)
    DISPLACE_X = np.sum(CLOSE_NEIGHBOURS * XX, axis=1)
    DISPLACE_Y = np.sum(CLOSE_NEIGHBOURS * YY, axis=1)

    # rule 3
    DELTAV_X = np.sum(Vx * NEIGHBOURS, axis=1) / NB_NEIGHBOURS
    DELTAV_Y = np.sum(Vy * NEIGHBOURS, axis=1) / NB_NEIGHBOURS
    DELTAV_X += np.sum(Vx * SELF, axis=1) / 8
    DELTAV_Y += np.sum(Vy * SELF, axis=1) / 8

    # boundaries
    BOUND_X = (X < Xmin) * 10 + (X > Xmax) * -10
    BOUND_Y = (Y < Ymin) * 10 + (Y > Ymax) * -10
    
    # update
    Vx += CENTER_OF_MASS_X + DISPLACE_X + DELTAV_X + BOUND_X
    Vy += CENTER_OF_MASS_Y + DISPLACE_Y + DELTAV_Y + BOUND_Y

    # speed limit
    Vx = (np.abs(Vx) > Vxmax) * Vx / np.linalg.norm(Vx) * Vxmax + (np.abs(Vx) <= Vxmax) * Vx
    Vy = (np.abs(Vy) > Vymax) * Vy / np.linalg.norm(Vy) * Vymax + (np.abs(Vy) <= Vymax) * Vy

    X += Vx
    Y += Vy

---

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [ ]:
def animate(t):
    global X, Y
    update()
    plt.scatter(X, Y)
    plt.xlim(Xmin, Xmax)
    plt.ylim(Ymin, Ymax)
    plt.show()

In [ ]:
widgets.interact(animate, t=widgets.Play())

---